Amazon Web Scrapper


In [6]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver 
from msedge.selenium_tools import Edge


def get_url(search_term):
    template = "https://www.amazon.com/s?k={}&crid=SZCW0NF454V2&sprefix=ultrawide+monit%2Caps%2C473&ref=nb_sb_noss_2"
    search_term = search_term.replace(" ", "+")

    #add search query to the url
    url = template.format(search_term)

    #add page query placeholder
    url += "$page{}"
    
    return url

def extract_record(item):
    """"Extract and return data from a single finding"""

    #scrapping description and url

    atag = item.h2.a
    description = atag.text.strip()
    url = "https://www.amazon.com/" + atag.get("href")
    
    try:
        #retriving price
        price_parent = item.find("span", "a-price")
        price = price_parent.find("span", "a-offscreen").text

    except AttributeError:

        return
        
    #retriving rating

    try:
        rating = item.i.text

    except AttributeError:

        rating = ""
    

    result = (description, price, rating, url)
    return result



def main(search_term):

    #staring the web driver
    edgeBrowser = webdriver.Edge(executable_path= r"C:\Users\kunwa\OneDrive\Documents\Kunwardeep\Python Scripts\WebScraping\edgedriver_win64\EdgeDriver.exe") # webdriver path
    
    
    url = get_url(search_term)

    
    records = []

    for page in range(1,21): #(1,21) defines the range of pages which are to be scrapped

        edgeBrowser.get(url.format(page))

        soup = BeautifulSoup(edgeBrowser.page_source, 'html.parser')

        results = soup.find_all("div", {"data-component-type": "s-search-result"})

        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    
    edgeBrowser.close()

    # Save the result to a .csv file

    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["Description", "Price", "Rating", "Url"])
        writer.writerows(records)


    

        


In [7]:
search_query = input("Specify the item: " )
main(search_query)
